In [1]:
import torch

n_quantiles = 3 

taus = torch.linspace(0.0, 1.0, steps=n_quantiles + 2)[1:-1] 

In [2]:
taus

tensor([0.2500, 0.5000, 0.7500])

In [3]:
preds = torch.tensor([
    [[1, 2, 4], [ 2, 3, 4], [1, 3, 5], [1, 2, 3]],
    [[2, 3, 4], [-1, 0, 1], [1, 3, 5], [1, 2, 5]]
], dtype=torch.float32)

print(preds.size(2))

3


In [6]:
taus

tensor([[[0.2500, 0.5000, 0.7500]]])

In [7]:
taus = torch.linspace(0.0, 1.0, steps=n_quantiles + 2)[1:-1].unsqueeze(0).unsqueeze(0)
td_error = [[[ 2.9700,  1.9600,  1.9400]],[[ 1.0000,  0.0000, -3.0000]]]
quantile_loss = torch.abs(taus - (td_error < 0).float())


TypeError: '<' not supported between instances of 'list' and 'int'

The following code is based on the tutorial give in https://www.kaggle.com/code/auxeno/quantile-regression-dqn-rl 

Some graphs and ideas should be included form the following link


Finally materials should be combined and written in the repo style to use advantages of both links! 

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import gymnasium as gym
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
import random 
import numpy as np 

class QRNetwrok(nn.modules):
    def __init__(self,observation_space,action_space,hidden_size = 64, num_quantiles = 16):
        self.input_size = np.prod(observation_space.shape)
        self.n_actions = action_space.n 
        self.n_quantiles = num_quantiles
        self.hidden_size = hidden_size

        self.input = nn.Linear(self.input_size , self.hidden_size)
        self.hidden = nn.Linear(self.hidden_size,self.hidden_size)
        self.out = nn.Linear(self.hidden_size , self.n_actions*self.n_quantiles)  

    
    def forward(self,x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        out = self.out(x)

        return out.view(-1,self.n_actions,self.n_quantiles)

# Action selection 

    def action_selection(self,observation , eps):
        if np.random.rand() < eps:
            return np.random.randint(0,self.n_actions)
        output = self.forward(torch.tensor(observation))
        return (output.mean(-1).argmax().item())
from collections import deque    
class ReplayBuffer:
    def __init__(self,maxlen):
        self.buffer = deque(maxlen)
    
    def put(self,experience):
        self.buffer.append(experience)
    
    def sample(self,batch_size):
        states , actions, rewards, next_states, dones = zip(*random.sample(self.buffer,batch_size))

    def __len__(self):
        return len(self.buffer)
    

class QRDQN():

    def __init__(self,config):
        self.config = config 

    def set_env(self,env):
        self.env = env 

    def initialize_agent(self):

        self.action_space = self.env.action_space  
        self.observation = self.env.observation_space 

        device = self.config['device'] 
        #def __init__(self,observation_space,action_space,hidden_size = 64, num_quantiles = 16)

        self.online_network = QRNetwrok(self.observation , 
                                        self.action_space, 
                                        self.config['hidden_size'], 
                                        self.config[n_quantiles]).to(device)
        
        self.target_network = QRNetwrok(self.observation , 
                                        self.action_space, 
                                        self.config['hidden_size'], 
                                        self.config[n_quantiles]).to(device)
        
        self.update_target_network(1.)

        self.optimizer = torch.optim.AdamW(self.online_network.parameter() ,lr = self.config['lr'] )


    def update_target_network(self,tau):
        with torch.no_grad():
            for target_params,online_params in zip(self.target_network.parameters()
                                                , self.online_network.parameters()):
                target_params.data.copy(tau*online_params.data + (1-tau)*target_params.data)

    def selection_action(self , observation, eps):
        if np.random.rand() < eps :
            #return np.random.randint(self.action_space.n)
            return self.env.action_space.sample()
        else : 
            output = self.online_network.forward(torch.tensor(observation))
            return output.mean(-1).argmax().index()
    
    def calc_exploration_rate(self,step,total_steps):
        epsilon_start = self.config['exploration_strat_eps']
        epsilon_end = self.config['exploration_end_eps']
        epsilon_decay_steps = total_steps * self.config['exploration_fraction']

        return max(epsilon_end ,
                   epsilon_start - (epsilon_start - epsilon_end) * (step / epsilon_decay_steps))
    
    def train(self, total_steps):

        episode_count = 0 
        episodic_reward = 0 
        episode_rewards = [np.NaN]

        observation , _ = self.env.reset()

        for step in (total_steps+1):
            


    

        


    




In [4]:
np.random.rand()

0.6601733860110626